# Train a gesture recognition model for microcontroller use

This notebook demonstrates how to train a 20kb gesture recognition model for [TensorFlow Lite for Microcontrollers](https://tensorflow.org/lite/microcontrollers/overview). It will produce the same model used in the [magic_wand](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/lite/micro/examples/magic_wand) example application.

The model is designed to be used with [Google Colaboratory](https://colab.research.google.com).

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/magic_wand/train/train_magic_wand_model.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/magic_wand/train/train_magic_wand_model.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


Training is much faster using GPU acceleration. Before you proceed, ensure you are using a GPU runtime by going to **Runtime -> Change runtime type** and selecting **GPU**. Training will take around 5 minutes on a GPU runtime.

## Configure dependencies

Run the following cell to ensure the correct version of TensorFlow is used.

We'll also clone the TensorFlow repository, which contains the training scripts, and copy them into our workspace.

In [3]:
# Clone the repository from GitHub
!git clone --depth 1 -q https://github.com/tensorflow/tensorflow
# Copy the training scripts into our workspace
!cp -r tensorflow/tensorflow/lite/micro/examples/magic_wand/train train
# Copy custom train.py to the train sub-folder
!cp train.py train

## Prepare the data

Next, we'll download the data and extract it into the expected location within the training scripts' directory.

In [4]:
# Download the data we will use to train the model
!wget http://download.tensorflow.org/models/tflite/magic_wand/data.tar.gz
# Extract the data into the train directory
!tar xvzf data.tar.gz -C train 1>/dev/null

--2020-07-18 19:26:46--  http://download.tensorflow.org/models/tflite/magic_wand/data.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 216.58.194.176, 2607:f8b0:4005:804::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|216.58.194.176|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 451862 (441K) [application/x-gzip]
Saving to: ‘data.tar.gz’

data.tar.gz         100%[===================>] 441.27K  --.-KB/s    in 0.07s   

2020-07-18 19:26:46 (5.86 MB/s) - ‘data.tar.gz’ saved [451862/451862]

tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
tar: Ign

We'll then run the scripts that split the data into training, validation, and test sets.

In [5]:
# The scripts must be run from within the train directory
%cd train
# Prepare the data
!python data_prepare.py
# Split the data by person
!python data_split_person.py

/tf/notebooks/train
data_length: 981
data_length:981
train_length:653
valid_length:136
test_length:192


## Load TensorBoard

Now, we set up TensorBoard so that we can graph our accuracy and loss as training proceeds.

In [6]:
# Load TensorBoard
%load_ext tensorboard
%tensorboard --logdir logs/scalars --host=0.0.0.0

## Begin training

The following cell will begin the training process. Training will take around 5 minutes on a GPU runtime. You'll see the metrics in TensorBoard after a few epochs.

In [ ]:
!python train.py --model CNN --person true

2020-07-18 19:26:58.486064: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-18 19:26:58.508044: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-18 19:26:58.508247: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1650 major: 7 minor: 5 memoryClockRate(GHz): 1.56
pciBusID: 0000:01:00.0
2020-07-18 19:26:58.508387: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-07-18 19:26:58.509205: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-07-18 19:26:58.509934: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.

Train for 1000 steps, validate for 5 steps
Epoch 1/50
2020-07-18 19:27:02.416876: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-07-18 19:27:02.559368: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-07-18 19:27:03.423120: I tensorflow/core/profiler/lib/profiler_session.cc:184] Profiler session started.
2020-07-18 19:27:03.423735: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcupti.so.10.0
1000/1000 [==============================] - 4s 4ms/step - loss: 8.7953 - accuracy: 0.3863 - val_loss: 0.8223 - val_accuracy: 0.5441
Epoch 2/50
1000/1000 [==============================] - 2s 2ms/step - loss: 1.0839 - accuracy: 0.4291 - val_loss: 0.7887 - val_accuracy: 0.5993
Epoch 3/50
1000/1000 [==============================] - 2s 2ms/step - loss: 1.0387 - accuracy: 0.4593 - val_loss: 0.7883 - val

tf.Tensor(
[[ 77   4   0   1]
 [  0  81   0   3]
 [  0   3  84   1]
 [  0   0   1 129]], shape=(4, 4), dtype=int32)
Loss 0.1388267402847608, Accuracy 0.9661458134651184
2020-07-18 19:28:59.308754: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-18 19:28:59.308961: I tensorflow/core/grappler/devices.cc:55] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2020-07-18 19:28:59.309022: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2020-07-18 19:28:59.309557: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-18 19:28:59.309731: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 165

2020-07-18 19:29:00.551568: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-18 19:29:00.551731: I tensorflow/core/grappler/devices.cc:55] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2020-07-18 19:29:00.551783: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2020-07-18 19:29:00.552269: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-18 19:29:00.552419: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1650 major: 7 minor: 5 memoryClockRate(GHz): 1.56
pciBusID: 0000:01:00.0
2020-07-18 19:29:00.552441: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Success

## Create a C source file

The `train.py` script writes a model, `model.tflite`, to the training scripts' directory.

In the following cell, we convert this model into a C++ source file we can use with TensorFlow Lite for Microcontrollers.

In [5]:
# Install xxd if it is not available
!apt-get -qq install xxd
# Save the file as a C source file
!xxd -i model.tflite > model.cc
# Print the source file
!cat model.cc

unsigned char model_tflite[] = {
  0x1c, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x12, 0x00,
  0x1c, 0x00, 0x04, 0x00, 0x08, 0x00, 0x0c, 0x00, 0x10, 0x00, 0x14, 0x00,
  0x00, 0x00, 0x18, 0x00, 0x12, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00,
  0x24, 0x4c, 0x00, 0x00, 0x10, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00,
  0x2c, 0x00, 0x00, 0x00, 0x0c, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x14, 0x01, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0xdc, 0x00, 0x00, 0x00,
  0x0f, 0x00, 0x00, 0x00, 0x54, 0x4f, 0x43, 0x4f, 0x20, 0x43, 0x6f, 0x6e,
  0x76, 0x65, 0x72, 0x74, 0x65, 0x64, 0x2e, 0x00, 0x12, 0x00, 0x00, 0x00,
  0xb0, 0x00, 0x00, 0x00, 0xa4, 0x00, 0x00, 0x00, 0x98, 0x00, 0x00, 0x00,
  0x8c, 0x00, 0x00, 0x00, 0x80, 0x00, 0x00, 0x00, 0x74, 0x00, 0x00, 0x00,
  0x68, 0x00, 0x00, 0x00, 0x60, 0x00, 0x00, 0x00, 0x58, 0x00, 0x00, 0x00,
  0x50, 0x00, 0x00, 0x00, 0x48, 0x00, 0x00, 0x00, 0x40, 0x00, 0x00, 0x00,
  0x38, 0x00, 0x00, 0x00, 0x2c, 0x00, 0x00, 0x00, 0x20, 0x00, 0x00, 0x00,
  0x1